# Project 2: Clustering + SVM to Predict Online Purchases
# DAV 6150

- Name: Zhengnan Li
- Repository: [Project_2/Z_Li_Project2.ipynb](https://github.com/Zhengnan817/DAV-6150/blob/5b0700d14267b43c664aca6d699e92ac88a638bc/Project_2/Z_Li_Project2.ipynb)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Introduction
For this Project, I will be working with a data set comprised of a variety of such web site metrics. My objective for Project 2 is to use clustering algorithms to create groupings of similar data observations within the provided data set, apply labels to the data observations assigned to those groupings, and then, after completing the necessary EDA and data prep work, construct an SVM model to predict the most likely categorization of any previously unseen data items. And as the data science practitioner, I will determine which features to include in my SVM models.  
In short, we will use the dataset to help the online retailers to try to determine whether or not a given site visitor will actually make a 
purchase.

### 1.1 Approach:
- 1. [Introduction](#1-Introduction): Import the data set and introduce data variables.  
- 2. [Pre-Clustering EDA](#2-Pre-Clustering-EDA): Explore the raw dataset and do analysis based on domain knowledge.  
- 3. [Pre-Clustering Data Preparation](#3-Pre-Clustering-Data-Preparation): Perform data cleaning, imputation and transformation.  
- 4. [Cluster Modeling](#4-Cluster-Modeling): Explain and present hierarchical and K-means clustering work.
- 5. [Post-Clustering Exploratory Data Analysis ](#5-Post-Clustering-Exploratory-Data-Analysis):Explain and present the post-clustering EDA work.
- 6. [Clustering Output vs. Actual Labels](#6-Clustering-Output-vs.-Actual-Labels): Compare the content of V_Revenue to the content of the Revenue column generated by clustering algorithm.
- 7. [SVM Modeling](#7-SVM-Modeling): SVM modeling work including any feature selection methods used and the use of any kernel functions.  
- 8. [Select Models](#8-Select-Models): Explain how I selected your model selection criteria and why. Also use the test dataset to predict.
- 9. [Clustering + SVM Output vs. Actual Labels](#9-Clustering-+-SVM-Output-vs.-Actual-Labels): Compare the content of V_Revenue to the 
content of the Revenue column generated by SVM algorithm.
- 10. [Conclusion](#10-Conclusion)

### 1.2 Data Introduction

In [2]:
online_shop = pd.read_csv("https://raw.githubusercontent.com/Zhengnan817/DAV-6150/main/Project_2/src/Project2_Data.csv")
online_shop.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend
0,0,0.0,0,0.0,8,222.000000,0.000000,0.028571,53.474571,0.0,May,1,1,1,2,New_Visitor,True
1,0,0.0,0,0.0,14,1037.500000,0.014286,0.047619,0.000000,0.0,Mar,2,2,4,2,Returning_Visitor,False
2,4,37.5,2,82.0,4,96.625000,0.000000,0.017500,0.000000,0.0,Nov,2,2,9,2,New_Visitor,False
3,4,115.7,0,0.0,16,655.383333,0.000000,0.012037,0.000000,0.0,Nov,1,1,2,3,Returning_Visitor,False
4,1,60.0,1,15.0,26,670.166667,0.000000,0.003846,0.000000,0.0,May,2,2,3,4,Returning_Visitor,False


In [3]:
online_shop.shape

(12330, 17)

In [4]:
online_shop.dtypes

Administrative               int64
Administrative_Duration    float64
Informational                int64
Informational_Duration     float64
ProductRelated               int64
ProductRelated_Duration    float64
BounceRates                float64
ExitRates                  float64
PageValues                 float64
SpecialDay                 float64
Month                       object
OperatingSystems             int64
Browser                      int64
Region                       int64
TrafficType                  int64
VisitorType                 object
Weekend                       bool
dtype: object

# 2. Pre Clustering EDA

# 3. Pre-Clustering Data Preparation

# 4. Cluster Modeling

# 5. Post-Clustering Exploratory Data Analysis

# 6. Clustering Output vs. Actual Labels

# 7. SVM Modeling

# 8. Select Models

# 9. Clustering + SVM Output vs. Actual Labels

# 10. Conclusion